# Introduction to the Multi-Agents



In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-preview"


#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Experimental.Agents;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
private const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";

// Agent is experimental
#pragma warning disable SKEXP0101
private readonly List<IAgent> s_agents = new();

private IAgent Track(IAgent agent)
{
    s_agents.Add(agent);

    return agent;
}

Installed Packages Microsoft.SemanticKernel, 1.3.0 Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-preview

In [2]:
#pragma warning disable SKEXP0101
private async Task<IAgent> CreateCocktailGeneratorAsync()
{
    // Initialize agent so that it may be automatically deleted.
    return
        Track(
            await new AgentBuilder()
                .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                .WithInstructions("Produce a cocktail name (no child elements) based on the given with ingredient at most 3 cocktails.")
                .WithName("Mixologist")
                .WithDescription("Generate 3 cocktails from ingredient.")
                .BuildAsync());
}

In [3]:
#pragma warning disable SKEXP0101
private async Task<IAgent> CreateJokeGeneratorAsync()
{
    // Initialize agent so that it may be automatically deleted.
    return
        Track(
            await new AgentBuilder()
                .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                .WithInstructions("Generate a funny joke about the given topic.")
                .WithName("Joker")
                .WithDescription("Generate a joke.")
                .BuildAsync());
}

In [4]:
#pragma warning disable SKEXP0101
private async Task<IAgent> CreateBarJokesGeneratorAsync()
{
    // Initialize the outline agent
    var cocktailGenerator = await CreateCocktailGeneratorAsync();
    // Initialize the research agent
    var jokeGenerator = await CreateJokeGeneratorAsync();

    // Initialize agent so that it may be automatically deleted.
    return
        Track(
            await new AgentBuilder()
                .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                .WithInstructions("You are bartender that tells jokes to customers. Use cocktail generator given an ingredient. Write a joke related to each cocktail")
                .WithName("Bartender")
                .WithDescription("Bartender that tells cocktail jokes on a given ingredient.")
                .WithPlugin(cocktailGenerator.AsPlugin())
                .WithPlugin(jokeGenerator.AsPlugin())
                .BuildAsync());
}

In [5]:
#pragma warning disable SKEXP0101
try
{
    // Initialize the agent with tools
    IAgent barJokesGenerator = await CreateBarJokesGeneratorAsync();

    // "Stream" messages as they become available
    await foreach (IChatMessage message in barJokesGenerator.InvokeAsync("vodka"))
    {
        Console.WriteLine($"[{message.Id}]");
        Console.WriteLine($"# {message.Role}: {message.Content}");
    }
}
finally
{
    await Task.WhenAll(s_agents.Select(a => a.DeleteAsync()));
}

[msg_KfsIsYpku90rVLtJFVsOHQGs]
# user: vodka
[msg_uBPyjDS6m0XMal4syhr8Wy7b]
# assistant: Alright, gather 'round for some spirited humor! Let's start off with a classic:

"Why don't you ever see a Moscow Mule at a track meet?
Because every time it hears 'On your marks, Get set, Vodka!' it just kicks back and enjoys the race! 🥂🐴"

Now, for the suave Vodka Martini fans:
"Why did the vodka martini break up with the olive?
Because every time they got together, they ended up in a twist, and the vodka said it just couldn't deal with the 'pitted' emotions any longer!"

And for the cosmopolitan out there:
"Why don't cosmopolitans ever get lost in space?
Because no matter where they go, they're always in the 'in' galaxy! 🍸🌌"

Keep 'em coming, and remember, laughter is the best chaser!
